In [262]:
import torch
import numpy as np
import pandas as pd
import torch.utils
from torch import nn

In [263]:
#Loading house prices training and test dataset
hptrain = pd.read_csv("/Users/bartoszgawin/Desktop/ML/Practise/PyTorch_Basic/Datasets/hptrain.csv",index_col=0)
hptest = pd.read_csv("Datasets/hptest.csv",index_col=0)

In [264]:
#Transforming data into tensor data format from pandas' dataframe
X_ols_train = torch.tensor(hptrain[['LotArea', 'OverallQual', 'OverallCond', 'MasVnrArea', 'TotalBsmtSF']].values,requires_grad=True,dtype=torch.float32)
X_ols_train = nn.functional.normalize(X_ols_train)
Y_ols_train = torch.tensor(hptrain[['SalePrice']].values,requires_grad=True,dtype=torch.float32)


#Splitting data into iterable batches
olstensor = torch.utils.data.TensorDataset(X_ols_train,Y_ols_train)
olsdataloader = torch.utils.data.DataLoader(olstensor, batch_size=20,shuffle=True)

In [265]:
#Specifying the paramenters of our model
olsnet = nn.Sequential(nn.Linear(X_ols_train.size()[1],Y_ols_train.size()[1]))
olsnet[0].weight.data.normal_(0, 0.1)
olsnet[0].bias.data.normal_(0, 0.1)
olsloss = nn.MSELoss()
olstrainer = torch.optim.SGD(olsnet.parameters(),lr=1)

In [266]:
num_epochs = 10
olsnet.train()
for epoch in range(num_epochs):
    print('-------------------')
    print(f'{epoch+1}th epoch')
    print('===================')
    olstrainer.zero_grad()
    X, Y = next(iter(olsdataloader))
    output = olsnet(X)

    l = olsloss(output,Y)
    print(f'Loss ols:{l}')


    print(f'Grad bef:{l.grad}')
    l.backward(retain_graph=True)
    print(f'Grad aft:{l.grad}')

    olstrainer.step()
    print('Coefficients: ',olsnet[0].weight.data)


-------------------
1th epoch
Loss ols:32001507328.0
Grad bef:32001507328.0
Grad aft:None
Coefficients:  tensor([[3.3743e+05, 2.6395e+02, 2.3692e+02, 3.5633e+03, 4.3951e+04]])
-------------------
2th epoch
Loss ols:265433481216.0
Grad bef:265433481216.0
Grad aft:None
Coefficients:  tensor([[-6.7656e+05, -6.5304e+02, -5.3736e+02, -2.3097e+04, -7.9519e+04]])
-------------------
3th epoch
Loss ols:2370302377984.0
Grad bef:2370302377984.0
Grad aft:None
Coefficients:  tensor([[ 2.3781e+06,  1.1722e+03,  1.1978e+03, -3.3014e+03,  2.4392e+05]])
-------------------
4th epoch
Loss ols:21014788440064.0
Grad bef:21014788440064.0
Grad aft:None
Coefficients:  tensor([[-6.7383e+06, -4.6216e+03, -4.2228e+03, -3.8503e+04, -6.7780e+05]])
-------------------
5th epoch
Loss ols:188032812580864.0
Grad bef:188032812580864.0
Grad aft:None
Coefficients:  tensor([[2.0426e+07, 1.5221e+04, 1.4547e+04, 2.8508e+05, 2.6730e+06]])
-------------------
6th epoch
Loss ols:nan
Grad bef:nan
Grad aft:None
Coefficients:  

/Users/bartoszgawin/lib/python3.10/site-packages/torch/_tensor.py:1083: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at  /Users/runner/work/pytorch/pytorch/pytorch/build/aten/src/ATen/core/TensorBody.h:482.)
  return self._grad
